In [66]:
import sys
import json
import datetime
from datetime import timedelta
import dateutil.parser
import random
import pandas as pdo

In [67]:
lookup_cluster = pd.read_csv('lookup.csv')

In [68]:
def trace_weekday(weekday_num):
    weekday_lst = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    filename = 'lookup_' + weekday_lst[weekday_num] + '.csv'
    return filename

In [69]:
def play_turn(current_datetime, currentCell):
    current_datetime = pd.to_datetime(current_datetime)
    current_hour = current_datetime.dt.hour
    current_day = current_datetime.dt.weekday
    currentCluster = lookup_cluster[lookup_cluster['cell']==str(currentCell)]['cluster']
    db = trace_weekday(current_day) # current weekday lookup table
    clusterdensity = pd.read_csv(db)
    target_cluster = clusterdensity[currentCluster] # current weekday , current cluster
    
    if (target_cluster[current_hour*3-1] == 'high'): 
        # suppose staying at current cell
        next_move = currentCell
    elif ((target_cluster[current_hour*3-1] == 'median')| 
         (target_cluster[current_hour*3-1] == 'low')): 
        # suppose going to the the max trip rate at current hour, highest cluster
        max_trip_rate = clusterdensity[current_hour*3-2].max()
        clusterdensity.loc[clusterdensity[current_hour*3-2] == max_trip_rate][current_hour*3]
   
    print("NextMove:"+next_move)
    return {"state":"FORHIRE","action":"MOVE","moveTo":next_move}

In [70]:
def first_move(start_datetime):
    start_datetime = start_datetime + timedelta(hours=12) 
    start_hour = start_datetime.dt.hour
    clusterdensity = lookup_density[(lookup_density['cluster'] == currentCluster)]
    next_move = lookup_density[max(clusterdensity[start_hour*3-2])][start_hour*3]
    return {"defer":start_datetime,"moveTo":next_move}

In [71]:
def next_shift(current_time):
    current_day = current_datetime.dt.weekday
    if (current_day == 0):  # if is Monday
        start_datetime = current_time + timedelta(hours=14)
        clusterdensity = lookup_density[(lookup_density['cluster'] == currentCluster)]
        next_move = lookup_density[max(clusterdensity[start_hour*3-2])][start_hour*3]
    
    if (current_day == 1): # if is Tuesday
        start_datetime = current_time + timedelta(hours=38)
        clusterdensity = lookup_density[(lookup_density['cluster'] == currentCluster)]
        next_move = lookup_density[max(clusterdensity[start_hour*3-2])][start_hour*3]
        
    if (current_day == 2): # if is Wednesday
        start_datetime = current_time + timedelta(hours=14)
        clusterdensity = lookup_density[(lookup_density['cluster'] == currentCluster)]
        next_move = lookup_density[max(clusterdensity[start_hour*3-2])][start_hour*3]
        
    if (current_day == 3):# if is Thursday
        start_datetime = current_time + timedelta(hours=14)
        clusterdensity = lookup_density[(lookup_density['cluster'] == currentCluster)]
        next_move = lookup_density[max(clusterdensity[start_hour*3-2])][start_hour*3]
    
    if (current_day == 4): # if is Friday
        start_datetime = current_time + timedelta(hours=13)
        clusterdensity = lookup_density[(lookup_density['cluster'] == currentCluster)]
        next_move = lookup_density[max(clusterdensity[start_hour*3-2])][start_hour*3]
        
    if (current_day == 5): # if is Saturday
        start_datetime = current_time + timedelta(hours=9)
        clusterdensity = lookup_density[(lookup_density['cluster'] == currentCluster)]
        next_move = lookup_density[max(clusterdensity[start_hour*3-2])][start_hour*3]
        
    if (current_day == 6): # if is Sunday
        start_datetime = current_time + timedelta(hours=14)
        clusterdensity = lookup_density[(lookup_density['cluster'] == currentCluster)]
        next_move = lookup_density[max(clusterdensity[start_hour*3-2])][start_hour*3]
        
    return {"defer":start_datetime,"moveTo":next_move}

In [72]:
while(1) :
    req = json.loads(sys.stdin.readline())
    reqtype = req['type']
    current_datetime = dateutil.parser.parse(req['time'])
    if(reqtype=="FIRSTMOVE"):
        action = first_move(current_datetime)
        #Create response named list that will be converted to JSON
        resp = {}
        resp["defer"]=action['defer'].strftime("%Y-%m-%dT%H:%M")
        resp["nextMove"]="REPOSITION"
        resp["moveTo"]=action['moveTo']
        print("MAST30034:" + json.dumps(resp))
        sys.stdout.flush()
    elif(reqtype=="PLAYTURN"):
        current_cell = req['currentCell']
        neighbours = req['neighbours']
        action=play_turn(current_datetime, current_cell, neighbours)
        print("MAST30034:" + json.dumps(action))
        sys.stdout.flush()
    elif(reqtype=="NEXTSHIFT"):
        action=next_shift(current_datetime)
        #Create response named list that will be converted to JSON
        resp = {}
        resp["defer"]=action['defer'].strftime("%Y-%m-%dT%H:%M")
        resp["nextMove"]="REPOSITION"
        resp["moveTo"]=action['moveTo']
        print("MAST30034:" + json.dumps(resp))
        sys.stdout.flush()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)